In [8]:
today_date = ''

StatementMeta(, 16f9a453-e7da-432b-b14d-ee95d009fd86, 10, Finished, Available, Finished)

In [3]:
abfs_path = 'abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Files/Landing/NycTaxiyellow'
partition_path = f"/processing_date={today_date}"
complete_path = abfs_path + partition_path
print(complete_path)



StatementMeta(, 16f9a453-e7da-432b-b14d-ee95d009fd86, 5, Finished, Available, Finished)

abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Files/Landing/NycTaxiyellow/processing_date=2026-01-13


In [4]:
from pyspark.sql.functions import col, to_date, col, trim, regexp_replace, when, expr, to_timestamp, date_format, lit, isnull, hour, concat_ws, sha2, dayofweek

nyctaxiyellow_landing_path = "abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Files/Landing/NycTaxiyellow"

df_yellow = spark.read.format('csv').option("header", "true").load(nyctaxiyellow_landing_path).filter(col('processing_date')==str(today_date))

StatementMeta(, 16f9a453-e7da-432b-b14d-ee95d009fd86, 6, Finished, Available, Finished)

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS nyctaxiyellow_bronze (
    VendorID string,
    tpep_pickup_datetime string,
    tpep_dropoff_datetime string,
    passenger_count string,
    trip_distance string,
    RatecodeID string,
    store_and_fwd_flag string,
    PULocationID string,
    DOLocationID string,
    payment_type string,
    fare_amount string,
    extra string,
    mta_tax string,
    tip_amount string,
    tolls_amount string,
    improvement_surcharge string,
    total_amount string,
    congestion_surcharge string,
    Airport_fee string,
    cbd_congestion_fee string,
    processing_date date
)

USING DELTA
PARTITIONED BY (processing_date);

StatementMeta(, 16f9a453-e7da-432b-b14d-ee95d009fd86, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
from pyspark.sql.functions import col, lit
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

(
    df_yellow
    .withColumn("processing_date", lit(today_date))
    .write
    .format("csv")
    .mode("overwrite")
    .option("header", "true")
    .partitionBy("processing_date")
    .saveAsTable("nyctaxiyellow_bronze")
)

StatementMeta(, 16f9a453-e7da-432b-b14d-ee95d009fd86, 8, Finished, Available, Finished)

In [7]:
df = spark.sql("SELECT * FROM NycTaxiProject.nyctaxiyellow_bronze")
df.count()

StatementMeta(, 16f9a453-e7da-432b-b14d-ee95d009fd86, 9, Finished, Available, Finished)

4591845